# RAG System for Survey Variable Resolution

This Jupyter Notebook is designed to implement a RAG (Retrieval-Augmented Generation) system. The primary purpose of this system is to resolve user-submitted survey variable data and find other variables that are likely to match.

## Overview

The RAG system leverages the power of retrieval-based and generative models for machine learning. It uses a two-step process:

1. **Retrieval**: The system retrieves relevant documents (in this case, survey variables) from a Pinecone DB knowledge source based on the user-submitted data.

2. **Generation**: The system then uses the retrieved documents to generate a response using a Cohere Reranking LLM and OpenAI's ChatOpenAI GPT 3.5 model. This response includes other variables that have a high likelihood of matching the user-submitted data.

## Usage

To use this notebook, input your survey variable data using the `input_data` directory. The RAG system will process this data, retrieve relevant variables from the knowledge source, and generate a list of variables that are likely to match your input.

## Benefits

The RAG system provides a powerful tool for survey data analysis. It can help identify patterns and correlations in the data, which can be invaluable for research and decision-making.

Please note that the accuracy of the system's output depends on the quality and comprehensiveness of the knowledge source. Therefore, it's crucial to continually update and expand the knowledge source to improve the system's performance.

## Setup

In [14]:
import os
import json
import pandas as pd
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.retrievers.multi_query import MultiQueryRetriever

import cohere
from pinecone import Pinecone
from pinecone import ServerlessSpec



from getpass import getpass
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings


In [ ]:
import os
from getpass import getpass
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

#### API KEYS

In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or getpass("OpenAI API Key: ")
COHERE_API_KEY = os.environ['COHERE_API_KEY']
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") or getpass("Enter your Pinecone API key: ")



#### Configuration Values

In [15]:
PINECONE_INDEX = "langchain-multi-query-demo" # Name of the Pinecone index
TEMPERATURE = 0
VECTOR_TEXT_FIELD = "text"
EMBEDDING_MODEL = "text-embedding-ada-002"


#### Logging

In [12]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

### Embedding and Vector DB Setup

In [6]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
spec = ServerlessSpec(
    cloud="aws", region="us-west-2"
)

In [16]:
import time

index_name = PINECONE_INDEX
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
  raise Exception("Pinecone index does not exist")

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 388951}},
 'total_vector_count': 388951}

In [17]:
embed = OpenAIEmbeddings(
    model=EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY, disallowed_special=()
)

In [19]:
from langchain.vectorstores import Pinecone

vectorstore = Pinecone(index, embed.embed_query, VECTOR_TEXT_FIELD)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


## Parse User Input

In [3]:
# Specify the path to your CSV file
csv_file_path = 'input_data/test_data_dictionary.csv'

# Read the CSV file and convert it into a DataFrame
df = pd.read_csv(csv_file_path)

In [4]:
df.describe()

,Sample/Subject Count
count,336.000000
mean,7418.211310
std,3661.967493
min,12.000000
25%,4388.000000
50%,10122.000000
75%,10370.000000
max,12240.000000


In [5]:
df = df.head(5)
# The column we need to input into the RAG is 'description'
# While in test mode this is all we want to do

In [20]:
df['description']

0            Consent group as determined by DAC
1    Source repository where subjects originate
2      Subject ID used in the Source Repository
3                              Affection status
4     Source repository where samples originate
Name: description, dtype: object

## Setup Retriever

We'll try this with two prompts, both encourage more variety in search queries.

**Prompt A**
```
Your task is to generate 3 different search queries that aim to
answer the user question from multiple perspectives.
Each query MUST tackle the question from a different viewpoint,
we want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.
Original question: {question}
```


**Prompt B**
```
Your task is to compare the results a variable lookup and rerankings
with the query Data Dictionary. The user would like to know how
closely each of their varaiables aligns with varaibles in the data corpus.
In order to do this, we need to compare the results of the original
query for each variable with the results from the variable lookup.
If the variable has a high similarity with the results from the variable
lookup, then we should say it is a good match. With variables that have no match,
we should say that there is no match. If the variable has a low similarity with the
results from the variable lookup, then we should discuss the matches
that result and how the user may address this variable for better alignment.
We want to provide the user with as much information as we can on how to
align their variables with the data corpus.
Original question: {question}
```

In [7]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [9]:
# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)

output_parser = LineListOutputParser()


### Prompt Template

In [ ]:

template = """
Your task is to compare the results a variable lookup and rerankings
with the query Data Dictionary. The user would like to know how
closely each of their varaiables aligns with varaibles in the data corpus.
In order to do this, we need to compare the results of the original
query for each variable with the results from the variable lookup.
If the variable has a high similarity with the results from the variable
lookup, then we should say it is a good match. With variables that have no match,
we should say that there is no match. If the variable has a low similarity with the
results from the variable lookup, then we should discuss the matches
that result and how the user may address this variable for better alignment.
We want to provide the user with as much information as we can on how to
align their variables with the data corpus.
Original question: {question}
"""

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template=template,
)
llm = ChatOpenAI(temperature=TEMPERATURE, openai_api_key=OPENAI_API_KEY)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

In [ ]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

## Connecting RAG elements

In [ ]:
from langchain.chains import TransformChain

def retrieval_transform(inputs: dict) -> dict:
    docs = retriever.get_relevant_documents(query=inputs["question"])
    docs = [d.page_content for d in docs]
    docs_dict = {
        "query": inputs["question"],
        "contexts": "\n---\n".join(docs)
    }
    return docs_dict

retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts"],
    transform=retrieval_transform
)